In [11]:
#Stock Investing with Python Part2 
#=========Event Analyzer=========
#Kun Zhuang
#iamzjk@hotmail.com

In [12]:
import pandas as pd
import numpy as np
import datetime as dt
import math
import copy
import QSTK.qstkutil.qsdateutil as du
import QSTK.qstkutil.DataAccess as da
import QSTK.qstkutil.tsutil as tu
import QSTK.qstkstudy.EventProfiler as ep

In [13]:
# find_events function
def find_events(symbols,startdate,enddate):
    '''symbols is either 'sp5002008' or 'sp5002012', date as list [year,month,day]'''
    print 'Accessing Yahoo dataset'
    dataobj = da.DataAccess('Yahoo')
    print 'Getting symbols'
    ls_symbols = dataobj.get_symbols_from_list(symbols)
    ls_symbols.append('SPY')
    print 'creating timestamps'
    dt_start = dt.datetime(startdate[0],startdate[1],startdate[2])
    dt_end = dt.datetime(enddate[0],enddate[1],enddate[2])
    ldt_timestamps = du.getNYSEdays(dt_start,dt_end,dt.timedelta(hours=16))
    print 'Downloading data from Yahoo'
    ls_keys = ['open', 'high', 'low', 'close', 'volume', 'actual_close']
    df_data = dataobj.get_data(ldt_timestamps,ls_symbols,ls_keys)
    print 'zipping data'
    d_data = dict(zip(ls_keys,df_data))
    print 'fill up NANs'
    for s_key in ls_keys:
        d_data[s_key] = d_data[s_key].fillna(method='ffill')
        d_data[s_key] = d_data[s_key].fillna(method='bfill')
        d_data[s_key] = d_data[s_key].fillna(1.0)
    print 'finding events'
    df_actual = d_data['actual_close']               #draw actual close price only
    df_events = copy.deepcopy(df_actual) * np.NAN     #creat event dataframe with NAN value in the same size as df_actual
    d_index = df_actual.index
    for sym in ls_symbols:
        for i in range(1,len(d_index)):
            if (df_actual[sym].ix[i-1] - df_actual[sym].ix[i])/df_actual[sym].ix[i-1] > 0.15:
#            if df_actual[sym].ix[i-1] >= 10.0 and df_actual[sym].ix[i] < 10.0:
                df_events[sym].ix[d_index[i]] = 1
    print 'finishing'
    return (df_events,d_data,ldt_timestamps,d_index)

In [14]:
# inputs
symbols1 = 'sp5002012'
startdate1 = [2008,1,1]
enddate1 = [2009,12,31]

In [15]:
# use function to get results
df_events,d_data,ldt_timestamps,d_index = find_events(symbols1,startdate1,enddate1)

Accessing Yahoo dataset
Getting symbols
creating timestamps
zipping data
fill up NANs
finding events
finishing


In [16]:
ep.eventprofiler(df_events, d_data, i_lookback=20, i_lookforward=20,
                s_filename='MyEventStudy_10_2012_15pct.pdf', b_market_neutral=True, b_errorbars=True,
                s_market_sym='SPY')

In [17]:
ldt_timestamps[10].date().year

2008